# Python and performance

In this post we discuss various aspects of performance when coding in Python.

## Compiled vs interpreted languages

Compiled languages require the use of a *compiler*. This is a computer program which converts the human-readable *source file* written by a human into machine-readable code. The output is an *executable file*, wwhich is a sequence of instruction which the central processing unit (CPU) of the computer is able to process and execute directly. Modern CPUs can execute billions of these elementary instructions every second. Because the instructions can directly be processed by the CPU, the executable is *architecture-dependent*. In other words, an executable compiled to run on your laptop will probably not run on your phone. The most common architectures today are x86-64 for computers and ARM for mobile and embedded devices. 

C++ is an ubiquitous compiled programming language; it was created to add object-oriented programming functionalities to C, which is even more widely used. Let's look at a simple sample C++ file, and its translation into x86-64 code.

In [ ]:
!cat example0.cpp

This program defines three variables `a`, `b` and `c`, and stores the sum of `a` and `b` into `c`. It then displays the result. To compile it, we'll use `gcc`, the GNU Compiler Collection.

In [ ]:
!g++ example0.cpp -o example0

Let's run the compiled executable:

In [ ]:
!./example0

A *disassembler* decomposes a binary executable into individual instructions for humans to read (this may look like a simple task, but is not an exact science!). Let's disassemble the executable we just created.

In [ ]:
!objdump -d example0

There is a lot there, but we are only interested in the section starting with `<main>:` since it directly corresponds to the C++ code we wrote. Let's isolate that part.

In [ ]:
!objdump -d example0 | sed -n '/<main>:/,/retq/p'

It's possible to recognise what we wrote in C++ after a while... for example, lines 908, 90f and 916 define the three variables, lines 91d and 920 push the contents of the variables to the CPU registers, and finally line 923 adds the value of the two registers! For more complicated programmes, this becomes tricky to follow, to say the least... but this is also the point: this code is only supposed to be read by a machine. However, we do see that the critical part of our programme was converted to a handful of CPU instructions, which is great for performance. If you'd like to get more hands-on with assembly code, here are two useful resources:
* https://www.agner.org/optimize: contains, amongst other things, a guide on optimization for x-86 processors (AMD, VIA and Intel). If you ever wondered what `mov`, `1ea` or `callq` do, what their throughput and latencies are on a given CPU, this is the place to go.
* Intel publishes "intrinsics" i.e. C functions that map directly to individual CPU instructions. More here: https://software.intel.com/sites/landingpage/IntrinsicsGuide/#

Interpreted languages, function differently from compiled ones. Instead of being converted to CPU instructions directly, the source code is parsed then intermediate, machine-independent *opcode* (or *bytecode*) is generated. This intermediate code is then *interpreted* by an interpreter (for the Python language, the reference interpreter is CPython - so called because it is written in C!). In a way, the interpreter is a "machine within the machine": it takes the place of the CPU (though of course it itself is being run by the CPU!). This explains some of the performance loss incurred when running Python scripts, since the presence of an interpreter induces overhead. We've written a python program equivalent to the C++ one above:

In [ ]:
!cat example1.py

In [ ]:
!python ./example1.py

One can explicitly ask the Python executable to generate the intermediate opcode. Ever wondered what the `.pyc` files present in `__pycache__` were?

In [ ]:
!python -m compileall .

In [ ]:
!python view_bytecode.py __pycache__/example1.cpython-36.pyc

Each group of opcodes directly corresponds to each line in `example1.py` (and this is much more readable than assembly code...). To execute each opcode, many CPU instructions are required - but how many? To get a feel for this, let's inspect the source code of the CPython interpreter! In fact, let's just consider the crucial opcode, which is `BINARY_ADD` (adds two numbers together). We'll have to clone the cpython repository... exciting!

In [ ]:
!git clone https://github.com/python/cpython.git

Feel free to have a closer look at ceval.c. We'll only show the relevant parts, since it is a lot of C code to take in in one go. Effectively, you'll find a big `switch` statement, followed by a lot of `case` statements, each corresponding to one opcode:

In [ ]:
!cat cpython/Python/ceval.c | grep 'case TARGET' | tail -20

Although this is C code, it's not hard to see what this is doing: the programme is looping over each opcode, and based on the value of the opcode does someting different. So what happens when `BINARY_ADD` is encountered?...

In [ ]:
!cat cpython/Python/ceval.c | sed -n '/BINARY_ADD/,/DISPATCH()/p'

Looks fairly complicated for a simple sum... but really all this does is call PyNumber_Add provided the arguments are not two Unicode strings (in which case they will be concatenated). So let's have a look at `PyNumber_Add`...

In [ ]:
!cat cpython/Objects/abstract.c | sed -n '/PyNumber_Add/,/return result/p'

So now we call `binary_op1`. If the result is `Py_NotImplemented` we do... something, else we return the result. Fine, let's have a look at `binary_op1`.

In [ ]:
!cat cpython/Objects/abstract.c | sed -n '/binary_op1/,/Py_RETURN_NOTIMPLEMENTED/p'

This seems to be doing some type checking. If the object in question a "number type" (if so define `slotv` function...) and more checking, and finally if possible run slotv. And do make sure to check that the returned type is not `PyNotImplemented`, of course...

What's going on here is that Python is (very) dynamically typed: types are associated to *values* rather than *variables*. So whenever an opration is executed on an object, the interpreter has to check that the relevant object supports it. An take care of potential failures. Everything is an object in Python! What was one CPU instruction in our C++ example is quickly turning into many more when the Python interpreter is running the show. Because of the flexibility that Python offers (dynamic typing, etc), a lot of optimisations that are available to other interpreted languages (e.g. Java) are simply not available to it. That is the main reason why Python is so slow! It's common for a C++ version of a Python function to be 100x faster.

## Improving performance

There are a number of ways in which you can improve the performance of your Python code. Normally, your programmes will spend most of their time in the same portion of the code - we call these *hotspots*. It is these hotspots that really need to be sped up, and not the rest of your code. For example, if your programme consists in reading a file from disk, parsing it into a custom data structure, and then running some bespoke ML algorithm on that structure, then probably only the latter step needs optimisation. We will discuss three different ways of speeding up your code: Cython, Numba and writing custom extensions; we won't discuss some others like using a different Python interpreter such as PyPy or Jython.

### Cython

Cython is an *optimising static compiler* for the Python language and the *extended Cython language, which is a superset of the Python programming language*. What does that mean? Cython compiles Python code, instead of interpreting it. Second, it defines a set of annotations that you may use on your Python code. These annotations are not part of the Python language. Rather, they help Cython understand the types of your variables and the signatures of your functions. Really, they let you write C code with Python syntax. The corollary here is that to use Cython well, you need to understand C. In particular, Cython is most succesful at optimising your code when you do not use Python's convenient features like dynamically typed variables, introspection, and so on. Let's give Cython a go. To compare its performance with pure Python, we'll test a function adding the first $10^8$ integers and returns the output.

In [ ]:
!cat add_integers_python.py

Cython code should be written in files ending in '.pyx'

In [ ]:
!cat add_integers_cython.pyx

This is fairly similar to the pure Python function. The main difference is that we annoted the type of the variables using the `cdef` keyword, and also annotated the function arguments. Note that `uint64_t` stands for unsigned 64 bit integer. It can hold any non-negative integer between 0 and $2^{64} - 1$, inclusive. To compile `add_integers_cython.pyx` into a Python extension, we have written a custom `setup.py` file:

In [ ]:
!cat setup.py 

We build the extension as follows:

In [ ]:
!python setup.py build_ext --inplace

Let's compare the pure python implementation to the Cython one.

In [ ]:
import time

In [ ]:
def timefunc(func, *args, **kwargs):
    t0 = time.time()
    func(*args, **kwargs)
    t1 = time.time()
    return t1 - t0

In [ ]:
from add_integers_python import f

In [ ]:
n = 100000000

In [ ]:
timefunc(f, n)

In [ ]:
from add_integers_cython import f  as f_cython

In [ ]:
timefunc(f_cython, n)

That's a roughly 200x improvement! You may want to check out `add_integers_cython.c`. The `add_integers_cython.pyx` file was tranlated into this C source file, and the latter was then compiled using a C compiler. For more information on Cython, see https://cython.org/

### Numba

Numba is easier to use than Cython. Numba's premise is that rather then having you specify types manually, it will try to infer them at runtime for you. As a consequence, most of the time all you have to do is to decorate your functions to tell numba to try to attempt and optimise them. No need for a different language here.

In [ ]:
from numba import jit

In [ ]:
f_numba = jit(nopython=True)(f)

In [ ]:
f_numba(10)

In [ ]:
timefunc(f_numba, n)

Woah! That was fast. How is this possible? You may want to use the `inspect_asm()` or `inspect_llvm()` methods of `f_numba` to check your theory.

It is also possible, and sometimes necessary, to specify the signatures of your functions directly, rather than to let `numba` find out what they are on its own. That's how you do it:

In [ ]:
from numba import uint64

In [ ]:
f_numba_with_signature = jit(
    uint64(uint64),
    nopython=True
)(f)

In [ ]:
f_numba_with_signature(100)

Numba has (very basic) support for classes, too.

In [ ]:
from numba import jitclass
from collections import OrderedDict

In [ ]:
@jitclass(
    OrderedDict([("x", uint64), ("y", uint64)]),
)
class Point2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

Some trickery is required if you want to specify the signature of functions using your jitclassed classes...

In [ ]:
NumbaPoint2DType = Point2D.class_type.instance_type

@jit(
    NumbaPoint2DType(NumbaPoint2DType, NumbaPoint2DType),
    nopython=True
)
def add_points(a, b):
    return Point2D(a.x + b.x, a.y + b.y)

In [ ]:
result = add_points(Point2D(1, 2), Point2D(3, 4))

In [ ]:
(result.x, result.y)

Class support is limited, however. For example, it is not possible for a class to reference itself - something you would do using pointers in C++. So defining recursive structures is very difficult (I have seen one example online, and could not adapt it to my needs). Again, knowing some C is useful here to understand what can and cannot be done with `numba`. For more information, follow the documentation here: http://numba.pydata.org/

### Using C/C++ directly

If C knowledge is effectively required to properly use Cython or Numba, might we not want to code our functions directly in C? Indeed, this is possible, and is probably preferable. One of the risks in using Cython or Numba is that they may not always suit your needs (see limitations above). You would not want to realise this in the middle of a project...

#### The C Foreign Function Interface (`cffi`) package

#### Writing a C++ extension with Boost

#### Writing a C extension from scratch

## Parallelisation

### System processes and threads

### The `threading` package

### The `multiprocessing` package